# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json 
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
output_file = pd.read_csv("../output_data/weather_df.csv")
output_file.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,emba,99,KZ,1584643002,71,48.83,58.14,36.41,7.65
1,1,mataura,50,NZ,1584643003,68,-46.19,168.86,50.00,14.00
2,2,chuy,3,UY,1584643003,50,-33.70,-53.46,77.09,16.84
3,3,esperance,75,AU,1584643003,93,-33.87,121.90,59.00,4.70
4,4,kurilsk,79,RU,1584643004,99,45.23,147.88,36.05,7.31


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(api_key=g_key)


In [46]:
help(gmaps.heatmap_layer)

Help on function heatmap_layer in module gmaps.heatmap:

heatmap_layer(locations, weights=None, max_intensity=None, dissipating=True, point_radius=None, opacity=0.6, gradient=None)
    Create a heatmap layer.
    
    This returns a :class:`gmaps.Heatmap` or a :class:`gmaps.WeightedHeatmap`
    object that can be added to a :class:`gmaps.Figure` to draw a
    heatmap. A heatmap shows the density of points in or near
    a particular area.
    
    To set the parameters, pass them to the constructor or set
    them on the ``Heatmap`` object after construction::
    
    >>> heatmap = gmaps.heatmap_layer(locations, max_intensity=10)
    
    or::
    
    >>> heatmap = gmaps.heatmap_layer(locations)
    >>> heatmap.max_intensity = 10
    
    :Examples:
    
    >>> fig = gmaps.figure()
    >>> locations = [(46.1, 5.2), (46.2, 5.3), (46.3, 5.4)]
    >>> heatmap = gmaps.heatmap_layer(locations)
    >>> heatmap.max_intensity = 2
    >>> heatmap.point_radius = 3
    >>> heatmap.gradient = [

In [53]:
locations=output_file[["Lat", "Long"]]
weight = output_file["Humidity"]
figure = gmaps.figure()
heatmap=gmaps.heatmap_layer(locations, weight, 
                            dissipating=False,
                            max_intensity =1,
                            opacity = 0.5,
                            point_radius =1)
figure.add_layer(heatmap)

In [54]:
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
#narrow down data to: 
#max temperature <= 80 degrees but > 70
#wind speeds <10
#zero cloudiness
#drop na's
ideal_temps = (output_file["Max Temp"]<= 80) &( output_file["Max Temp"]> 70)
ideal_wind_spd = (output_file["Wind Speed"] < 10)
ideal_cloud_cov = (output_file["Cloudiness"] < 1)
ideal_weather = ideal_temps & ideal_wind_spd & ideal_cloud_cov
ideal_weather_df = output_file[ideal_weather]
ideal_weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
282,282,adrar,0,MR,1584643071,15,20.50,-10.07,79.43,9.31
291,291,khudaganj,0,IN,1584643073,41,28.15,79.72,70.25,5.23
399,399,richards bay,0,ZA,1584643098,75,-28.78,32.04,77.81,5.59
405,405,yulara,0,AU,1584643100,27,-25.24,130.99,77.00,6.93
474,474,marawi,0,PH,1584643116,85,8.00,124.29,78.01,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map